In [284]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import random
import urllib2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

First, define X

In [285]:
target_data = "./features.csv"
features = pd.read_csv(target_data, parse_dates = True)
features = features.drop(['Unnamed: 0'],axis=1)
#features = features.drop(['Operating Margin', 'P/B', 'ROA', 'EV', 'Shares Outstanding'],axis=1)
#features = features.drop(['Revenue'],axis=1)
print features.info()
print "--------------------------------------------------------"

X = []
for index, rows in features.iterrows():
    X.append([])
    X[index].append(1)
    for a in range(1, len(rows)):
        X[index].append(rows[a])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 26 columns):
Quote                       1459 non-null object
Beta                        1459 non-null float64
Current Ratio               1459 non-null float64
Diluted EPS                 1459 non-null float64
EBITDA                      1459 non-null float64
EV                          1459 non-null float64
Operating Margin            1459 non-null float64
P/B                         1459 non-null float64
P/S                         1459 non-null float64
Profit Margin               1459 non-null float64
Qualterly Revenue Growth    1459 non-null float64
ROA                         1459 non-null float64
ROE                         1459 non-null float64
Week Change                 1459 non-null float64
isBasicIndustries           1459 non-null int64
isCapitalGoods              1459 non-null int64
isConsumerDurables          1459 non-null int64
isConsumerNonDurables       1459 non-null int64
is

In [286]:
features.head()

,Quote,Beta,Current Ratio,Diluted EPS,EBITDA,EV,Operating Margin,P/B,P/S,Profit Margin,...,isConsumerDurables,isConsumerNonDurables,isConsumerServices,isEnergy,isFinance,isHealthCase,isMiscellaneous,isPublicUtilities,isTechnology,isTransportation
0,AABA,1.98,17.07,29.96,6.650000e+06,5.905000e+10,0.1221,0.95,1159.80,453.8760,...,0,0,0,0,0,0,0,0,1,0
1,AAL,1.27,0.61,3.90,6.790000e+09,4.170000e+10,0.1130,5.78,0.54,0.0455,...,0,0,0,0,0,0,0,0,0,1
2,AAME,0.77,1.21,0.20,8.520000e+06,7.366000e+07,0.0391,0.56,0.35,0.0250,...,0,0,0,0,1,0,0,0,0,0
3,AAOI,1.22,3.25,3.67,1.073300e+08,5.553100e+08,0.2274,1.65,1.44,0.1934,...,0,0,0,0,0,0,0,0,1,0
4,AAON,0.67,3.07,1.03,8.988000e+07,1.820000e+09,0.1848,7.99,4.68,0.1345,...,0,0,0,0,0,0,0,0,0,0


Define Y and filter out extra variables

In [287]:
target_data = "./stock01.csv"
stock01 = pd.read_csv(target_data, parse_dates = True)
stock01 = stock01.drop(['Unnamed: 0'],axis=1)
print stock01.info()
print "------------------------------------------------------"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 6 columns):
Stock              2660 non-null object
Bollinger Band     2660 non-null int64
Keltner Channel    2660 non-null int64
Moving Envelope    2660 non-null int64
Parabolic SAR      2660 non-null int64
Price Channel      2660 non-null int64
dtypes: int64(5), object(1)
memory usage: 124.8+ KB
None
------------------------------------------------------


In [288]:
overlays = pd.DataFrame({'Stock':features.Quote})
overlays.head()

,Stock
0,AABA
1,AAL
2,AAME
3,AAOI
4,AAON


In [289]:
overlays = pd.merge(stock01, overlays, on='Stock', how='inner')

In [290]:
overlays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 6 columns):
Stock              1459 non-null object
Bollinger Band     1459 non-null int64
Keltner Channel    1459 non-null int64
Moving Envelope    1459 non-null int64
Parabolic SAR      1459 non-null int64
Price Channel      1459 non-null int64
dtypes: int64(5), object(1)
memory usage: 79.8+ KB


In [291]:
y_pc = overlays['Price Channel'].tolist()
y_mae = overlays['Moving Envelope'].tolist()
y_bb = overlays['Parabolic SAR'].tolist()
y_kc = overlays['Keltner Channel'].tolist()
y_psar = overlays['Bollinger Band'].tolist()

Build the Logistic regression model

In [293]:
kfold = KFold(n_splits=10, random_state=5)

In [294]:
# The probability of price channels
X_train, X_test, y_train, y_test = train_test_split(X, y_pc, test_size=0.3, random_state=1)

lr_pc = LogisticRegression(solver='liblinear')
result_pc = lr_pc.fit(X_train, y_train)

print "Price Channels"
results = cross_val_score(lr_pc, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Price Channels
Average accuracy: 0.60920426423


In [295]:
# The probability of Moving Average Envelope
X_train, X_test, y_train, y_test = train_test_split(X, y_mae, test_size=0.3, random_state=1)

lr_mae = LogisticRegression(solver='liblinear')
result_mae= lr_mae.fit(X_train, y_train)

print "Moving Average Envelope"
results = cross_val_score(lr_mae, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Moving Average Envelope
Average accuracy: 0.67782219684


In [296]:
# The probability of Bolling Bands
X_train, X_test, y_train, y_test = train_test_split(X, y_bb, test_size=0.3, random_state=1)

lr_bb = LogisticRegression(solver='liblinear')
result_bb = lr_bb.fit(X_train, y_train)

print "Bollinger Bands"
results = cross_val_score(lr_bb, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Bollinger Bands
Average accuracy: 0.866771368742


In [297]:
# The probability of Keltner Channels
X_train, X_test, y_train, y_test = train_test_split(X, y_kc, test_size=0.3, random_state=1)

lr_kc = LogisticRegression(solver='liblinear')
result_kc = lr_kc.fit(X_train, y_train)

print "Keltner Channels"
results = cross_val_score(lr_kc, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Keltner Channels
Average accuracy: 0.865838568437


In [298]:
# The probability of Parabolic SAR
X_train, X_test, y_train, y_test = train_test_split(X, y_psar, test_size=0.3, random_state=1)

lr_psar = LogisticRegression(solver='liblinear')
result_psar = lr_psar.fit(X_train, y_train)

print "Parabolic SAR"
results = cross_val_score(lr_psar, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Parabolic SAR
Average accuracy: 0.857909765848


In [299]:
overlays['Bollinger Band'].value_counts()

0    1260
1     199
Name: Bollinger Band, dtype: int64

In [300]:
overlays['Keltner Channel'].value_counts()

0    1261
1     198
Name: Keltner Channel, dtype: int64

In [301]:
overlays['Moving Envelope'].value_counts()

0    1146
1     313
Name: Moving Envelope, dtype: int64

In [302]:
overlays['Parabolic SAR'].value_counts()

0    1255
1     204
Name: Parabolic SAR, dtype: int64

In [303]:
overlays['Price Channel'].value_counts()

0    914
1    545
Name: Price Channel, dtype: int64

It seems like we got much more price channels and moving average envelopes in our dataset, we try to decrease the number of price channels and moving average envelopes in our dataset.

In [304]:
#remove 345 Price Channels and 113 Moving Envelope

X_2 = []
y_pc_2 = []
y_mae_2 = []
y_kc_2 = []
y_bb_2 = []
y_psar_2 = []

In [305]:
num_pc = 0
num_mae = 0

index_list = np.arange(0, len(X))
random.shuffle(index_list)
print index_list

[ 691 1215 1369 ...,  755 1341  553]


In [306]:
for a in index_list:
    if(y_pc[a] == 1 and num_pc <= 200):
        X_2.append(X[a])
        y_pc_2.append(y_pc[a])
        y_mae_2.append(y_mae[a])
        y_kc_2.append(y_kc[a])
        y_bb_2.append(y_bb[a])
        y_psar_2.append(y_psar[a])
        num_pc += 1
    elif(y_pc[a] == 1 and num_pc > 200):
        continue
    elif(y_mae[a] == 1 and num_mae > 200):
        continue
    elif(y_mae[a] == 1 and num_mae <= 200 ):
        X_2.append(X[a])
        y_pc_2.append(y_pc[a])
        y_mae_2.append(y_mae[a])
        y_kc_2.append(y_kc[a])
        y_bb_2.append(y_bb[a])
        y_psar_2.append(y_psar[a])
        num_mae += 1
    else:
        X_2.append(X[a])
        y_pc_2.append(y_pc[a])
        y_mae_2.append(y_mae[a])
        y_kc_2.append(y_kc[a])
        y_bb_2.append(y_bb[a])
        y_psar_2.append(y_psar[a])

info = pd.DataFrame({'Price Channels':y_pc_2, 'Moving Envelope':y_mae_2, 'Keltner Channel':y_kc_2,
                    'Bollinger Band':y_bb_2, 'Parabolic SAR':y_psar_2})

In [307]:
info['Bollinger Band'].value_counts()

0    799
1    204
Name: Bollinger Band, dtype: int64

In [308]:
info['Keltner Channel'].value_counts()

0    805
1    198
Name: Keltner Channel, dtype: int64

In [309]:
info['Moving Envelope'].value_counts()

0    802
1    201
Name: Moving Envelope, dtype: int64

In [310]:
info['Parabolic SAR'].value_counts()

0    804
1    199
Name: Parabolic SAR, dtype: int64

In [311]:
info['Price Channels'].value_counts()

0    802
1    201
Name: Price Channels, dtype: int64

In [312]:
# The probability of price channels
X_train, X_test, y_train, y_test = train_test_split(X_2, y_pc_2, test_size=0.3, random_state=1)

lr_pc = LogisticRegression(solver='liblinear')
result_pc = lr_pc.fit(X_train, y_train)

print "Price Channels"
results = cross_val_score(lr_pc, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Price Channels
Average accuracy: 0.790543259557


In [313]:
# The probability of Moving Average Envelope
X_train, X_test, y_train, y_test = train_test_split(X_2, y_mae_2, test_size=0.3, random_state=1)

lr_mae = LogisticRegression(solver='liblinear')
result_mae= lr_mae.fit(X_train, y_train)

print "Moving Average Envelope"
results = cross_val_score(lr_mae, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Moving Average Envelope
Average accuracy: 0.709356136821


In [314]:
# The probability of Bolling Bands
X_train, X_test, y_train, y_test = train_test_split(X_2, y_bb_2, test_size=0.3, random_state=1)

lr_bb = LogisticRegression(solver='liblinear')
result_bb = lr_bb.fit(X_train, y_train)

print "Bollinger Bands"
results = cross_val_score(lr_bb, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Bollinger Bands
Average accuracy: 0.615573440644


In [315]:
# The probability of Keltner Channels
X_train, X_test, y_train, y_test = train_test_split(X_2, y_kc_2, test_size=0.3, random_state=1)

lr_kc = LogisticRegression(solver='liblinear')
result_kc = lr_kc.fit(X_train, y_train)

print "Keltner Channels"
results = cross_val_score(lr_kc, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Keltner Channels
Average accuracy: 0.802253521127


In [316]:
# The probability of Parabolic SAR
X_train, X_test, y_train, y_test = train_test_split(X_2, y_psar_2, test_size=0.3, random_state=1)

lr_psar = LogisticRegression(solver='liblinear')
result_psar = lr_psar.fit(X_train, y_train)

print "Parabolic SAR"
results = cross_val_score(lr_psar, X_train, y_train, cv=kfold, scoring='accuracy')
print("Average accuracy: {}".format(results.mean()))

Parabolic SAR
Average accuracy: 0.789175050302


The accuracy has deccreased a little bit after we decrease the number of price channels and moving average envolopes, and it seems more reasonable. So, we dicide to use model after decrease the number.